In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import cv2


2023-11-27 08:35:58.175401: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
interpreter = tf.lite.Interpreter(model_path="../models/3.tflite")
interpreter.allocate_tensors()


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [3]:
def draw_key_points(frame, keypoints, conf_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    i = 0
    for kp in shaped:
        ky, kx, kp_conf = kp
        if kp_conf > conf_threshold:
            cv2.circle(frame,(int(kx), int(ky)), 1, (0, 255, 0), 5)
            cv2.circle(frame,(int(10), int(10)), 2, (0, 255, 0), 5)
        if i == 0:
            print(ky, ky/720, kx, kx/1280)
        i += 1
    return frame


In [28]:
frame.shape


(720, 1280, 3)

In [8]:
cap = cv2.VideoCapture(0)
dynamic_variable = 0
while cap.isOpened():
    ret, frame = cap.read()

    img = frame.copy()
    img = cv2.resize(img, (192, 192))
    img = tf.image.resize_with_pad(np.expand_dims(img, axis=0), 192, 192)
    input_image = tf.cast(img, dtype=tf.float32)

    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()

    interpreter.set_tensor(input_details[0]["index"], np.array(input_image))
    interpreter.invoke()
    key_points_with_scores = interpreter.get_tensor(output_details[0]["index"])

    draw_key_points(frame, key_points_with_scores, conf_threshold=0.5)

    # Define the text to be displayed
    dynamic_variable += 1

    text = f"Variable Value: {dynamic_variable}"

    # Coordinates where the text will appear
    text_position = (50, 50)

    # Font settings
    font = cv2.FONT_HERSHEY_SIMPLEX
    font_scale = 1
    font_color = (255, 255, 255)  # White color
    line_type = 2

    # Put the text on the frame
    cv2.putText(frame, text, text_position, font, font_scale, font_color, line_type)

    cv2.imshow("Live Feed Test", frame)

    if cv2.waitKey(10) & 0xFF==ord("q"):
        break

cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)


395.004243850708 0.5486170053482056 839.6238708496094 0.6559561491012573
399.4052982330322 0.5547295808792114 833.0303192138672 0.6508049368858337
401.3541269302368 0.5574362874031067 833.1388854980469 0.6508897542953491
394.6677017211914 0.548149585723877 834.82666015625 0.6522083282470703
398.02711486816406 0.5528154373168945 834.1688537597656 0.6516944169998169
402.1346712112427 0.5585203766822815 834.1014862060547 0.6516417860984802
401.52501583099365 0.5576736330986023 833.9579010009766 0.6515296101570129
396.6679859161377 0.5509277582168579 828.4616851806641 0.6472356915473938
396.9211435317993 0.5512793660163879 829.0937805175781 0.6477295160293579
398.30177307128906 0.553196907043457 828.7586975097656 0.6474677324295044
397.2342538833618 0.5517142415046692 826.7800140380859 0.6459218859672546
394.1845178604126 0.5474784970283508 820.191650390625 0.6407747268676758
395.9401845932007 0.5499169230461121 818.8419342041016 0.6397202610969543
396.07579708099365 0.5501052737236023 821

-1